In [1]:
'''
Set path to the folder of used python files
'''
import sys
sys.path.insert(0, '/Users/lynnjiang/liuGit/Pybook/Clinic-Analysis/Scripts/Data_Cleaning')
from Abstract import Abstract

In [ ]:
import re
import numpy as np
import pandas as pd
class Prescriptions(Abstract):
    def __init__(self):
        super().__init__()

    def dosage_value(self, s):
        try:
            return float(s)
        except ValueError:
            return 0.0

    def read_prescriptions_data(self, prescriptions_file='PRESCRIPTIONS'):
        columns = ['SUBJECT_ID','DRUG','DOSE_VAL_RX','ENDDATE','STARTDATE']
        digit = re.compile('[^\d-]')

        prescriptions_df = self.read_data(prescriptions_file)
        pres_clean_df = prescriptions_df[columns].dropna(subset=['DOSE_VAL_RX'])

        ## Regularize value in ['DOSE_VAL_RX'] to numeric type, e.g. (300-600)->450
        pres_clean_df.DOSE_VAL_RX = pres_clean_df.DOSE_VAL_RX.apply(lambda x: np.average(list(map(self.dosage_value,digit.sub('',str(x)).split('-')))))

        return pres_clean_df
    
    def get_statistics(self,pres_df):
        return 0 
        

    def to_attributes(self, pres_df):
        ## Get period for patient taking a medicine in each record (ENDDATE-STARTDATE+1) (/day)
        enddate, startdate, period, ave_value = 'ENDDATE', 'STARTDATE', 'PERIOD', 'AVG_DOSE_VAL_RX'
        pres_df[enddate] = pd.to_datetime(pres_df[enddate])
        pres_df[startdate] = pd.to_datetime(pres_df[startdate])
        pres_df[period] = (pres_df[enddate] - pres_df[startdate]).dt.days + 1

#         ## Get average dosage for each patient taking each kind of drug
#         pres_sum_df = pres_df.groupby(['SUBJECT_ID','DRUG'])['DOSE_VAL_RX','PERIOD'].sum().reset_index()
#         pres_sum_df['AVG_DOSE_VAL_RX'] = round(pres_sum_df['DOSE_VAL_RX']/pres_sum_df['PERIOD'],6)

#         ## Transform dataframe to matrix (USER*DRUG)
#         user_prescription_vector = pd.pivot_table(pres_sum_df, index=['SUBJECT_ID'], columns=['DRUG'],\
#                                                   values=['AVG_DOSE_VAL_RX'])
#         ## Imputation: with 0
#         user_final_presvec= user_prescription_vector.fillna(0)
#         ## TODO: reset_index has some problems, the title is contained as a row
#         user_final_presvec = user_final_presvec.reset_index()
#         user_final_presvec.columns = user_final_presvec.columns.droplevel(-1)

        return pres_df


        ## Create binary value for diagnoses
        pres_clean_d['VALUE'] = 1
        user_pres_vec = pd.pivot_table(pres_clean_df, index=['SUBJECT_ID'], columns=['ICD9_CODE'],\
                                       values=['VALUE']).fillna(0)
        user_diag_vec = user_diag_vec.reset_index()
        self.write2file(user_diag_vec,'USER_VECTORS/diagnoses_uservectors')


## Get counts, max, min

In [ ]:
pp = Prescriptions()
pres_df = pp.read_prescriptions_data()
pres_df.head()

In [ ]:
stc_ls = ['min','max', 'mean','count']
pres_agg_df = pres_df.groupby(['SUBJECT_ID','DRUG'])['DOSE_VAL_RX'].agg(stc_ls)
pres_agg_df.head()

In [ ]:
# pres_agg_df['count'].max()
pres_agg_df=pres_agg_df.reset_index()
pres_vec = pres_agg_df.pivot(index='SUBJECT_ID', columns='DRUG')
pres_vec.head()

In [ ]:
pres_vec.reset_index()
pres_vec.columns = pres_vec.columns.droplevel(0)
pres_vec=pres_vec.fillna(0)
pres_vec.head()

In [2]:
pp= Abstract()
test = pp.read_data('temp/USER_VECTORS/prescriptions_uservectors')

In [4]:
test.shape

(39363, 17937)

## Following are drafts

In [ ]:
pp = Prescriptions()
user_list = pp.read_data('PATIENTS')['SUBJECT_ID']
pres_df = pp.to_attributes(pp.read_prescriptions_data(user_list))
# pp.write2file(user_vectors,'prestest_uservectors')

In [ ]:
pres_df[:20]

In [ ]:
pp.read_data('PRESCRIPTIONS')[:20]

In [ ]:
pres_df.groupby(['SUBJECT_ID','DRUG'])['DOSE_VAL_RX'].rolling(window=365, freq='D',min_periods=1).mean